## The 1D Wave Equation

$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$$

$$\text{Where $u$ is function of space and time, i.e:}\quad u(x,t) \quad $$

$$\text{Also, potentially $c(x)$ – but we'll begin with $c$ constant for normal mode solutions at first...}$$

#### Discretisation of 1d wave equation using 2nd-order finite-difference...

$$\text{Taylor series:} \quad f(x) = f(a)+{\frac {f'(a)}{1!}}(x-a)+{\frac {f''(a)}{2!}}(x-a)^{2}+{\frac {f'''(a)}{3!}}(x-a)^{3}+\cdots $$

$$\text{So:}\quad f(x_0+\delta x) = f(x_0)+{\frac {f'(x_0)}{1!}}\delta x+{\frac {f''(x_0)}{2!}}\delta x^{2}+{\frac {f'''(x_0)}{3!}}\delta x^{3}+O(\delta x^4)$$

$$\text{and:}\quad f(x_0-\delta x) = f(x_0)-{\frac {f'(x_0)}{1!}}\delta x+{\frac {f''(x_0)}{2!}}\delta x^{2}-{\frac {f'''(x_0)}{3!}}\delta x^{3}+O(\delta x^4)$$

$$\text{Combine above two:}\quad f(x_0+\delta x) + f(x_0-\delta x) = 2f(x_0) + f''(x_0)\delta x^{2} + O(\delta x^4)$$

$$\text{Rearrange:}\quad f''(x_0) = \frac{f(x_0+\delta x) + f(x_0-\delta x) - 2f(x_0)}{\delta x^2}\ +\ O(\delta x^2)$$

$$\text{Putting that into LHS of wave equation, to discretise time:}$$

$$\frac{u(x,t+\delta t) - 2u(x,t) + u(x,t-\delta t)}{\delta t^2} +O(\delta t^2) \ = \ c(x)^2\ \frac{\partial^2 u(x,t)}{\partial x^2}$$

$$\text{Which leads to time-stepping equation (using $\tau$ for the time index):}$$

$$\quad u^{\tau+1}(x)\ \approx \ \delta t^2c(x)^2\ \frac{\partial^2 u^\tau(x)}{\partial x^2} + 2u^\tau(x) - u^{\tau-1}(x)$$

$$\text{Now do the same for differential on RHS, discretising spatial dimension (using $\xi$ for $x$-position index):}$$

$$\quad u_\xi^{\tau+1}\ \approx \ \frac{\delta t^2c^2_\xi}{\delta x^2}\left(u_{\xi+1}^\tau-2u_\xi^\tau+u_{\xi-1}^\tau\right) + 2u_\xi^\tau - u_\xi^{\tau-1}$$

----
### Adding absorbing layer (in 1d)...

$$\text{Want wave to 'fade away' over time. So start from a constant frequency complex wave:}$$

$$u(x,t)\ =\ u_0(x)\ e^{-i\omega t}$$


$$\text{To make it fade away we can add an extra real term in the exponent:}\quad
u(x,t)\ =\ u_0(x)\ e^{-i\omega t - \alpha ct}$$

$$\text{That should cause it to decay by factor}\ \ e^{-\alpha ct}\ \ 
\text{as wave travels distance $ct$ from time zero to $t$.}$$

Let's see how this affects the wave equation...

$$\text{Differentiate w.r.t. $t$:}\quad \frac{\partial u}{\partial t}\ =\ -u_0(x)\ (i\omega+\alpha c)\ e^{-i\omega t - \alpha ct}\ =\ 
-i\omega\ u\ -\ \alpha c\ u$$

$$\text{And again:}\quad \frac{\partial^2 u}{\partial t^2}\ =\ u_0(x)\ (i\omega+\alpha c)^2\ e^{-i\omega t - \alpha ct}\ =\ 
-\omega^2\ u\ +\ \alpha^2 c^2\ u\ +\ 2i\omega\alpha c\ u$$

$$\text{But we can re-arrange the first equation for $\frac{\partial u}{\partial t}$ to give:}\quad
i\omega\ u\ =\ -\alpha c\ u - \frac{\partial u}{\partial t}$$

$$\text{So we have:}\quad\frac{\partial^2 u}{\partial t^2}\ =\ 
-\omega^2\ u\ +\ \alpha^2 c^2\ u\ -\ 2\alpha^2 c^2\ u\ -\ 2\alpha c\frac{\partial u}{\partial t}$$

$$\text{If we assume RHS of wave equation is unchanged (we only decayed with time), so still equals $-\omega^2 u$, then we have:}$$

$$\quad\frac{\partial^2 u}{\partial t^2}\ =\ 
c^2\frac{\partial^2 u}{\partial x^2} -\ \alpha^2 c^2\ u\ -\ 2\alpha c\frac{\partial u}{\partial t}$$

$$\text{Let's discretise the final term above in time as:}\quad
\left.\frac{\partial u}{\partial t}\right|_{t_\tau} \approx \frac{u^{\tau+1}(x)-u^{\tau-1}(x)}{2\delta t}$$

$$\text{Recall that our time-stepping equation before was:}\quad u^{\tau+1}(x)\ \approx \ 
\delta t^2c(x)^2\ \frac{\partial^2 u^\tau(x)}{\partial x^2} + 2u^\tau(x) - u^{\tau-1}(x)$$

$$\text{With the modification, it becomes:}\quad (1+q)\ u^{\tau+1}(x)\ \approx \ 
\delta t^2c(x)^2\ \frac{\partial^2 u^\tau(x)}{\partial x^2}\ +\ (2+q^2)\ u^\tau(x)\ -\ 
(1-q)\ u^{\tau-1}(x)$$

$$\text{Where:}\quad q=\alpha c\delta t\quad
\text{(note that it collapses back to original when we put $q$=0, as we would expect)}$$

Above change means we take our original propagation kernel code, and multiply the oldest wavefield, $u^{\tau-1}$, by $(1$$-$$q)$, and multiply the current wavefield, $u^\tau$, by $(2$+$q^2)$ instead of just 2 (though that makes little difference in practice, since $q$ is pretty small), and then the whole of RHS gets divided by $1$+$q$.

– It also ends up being just the same changes for the absorbing layer modifications in 2d.

#### Implementing absorbing layer in practice...

Note that $q$ above depends on our model, $c$, which can change over space. Also, we will find that we want the $\alpha$ to go from zero (across central main portion of domain where we want no absorption), and 'ramp up' as we move towards the boundaries – we'll try linear ramp, and show quadratic does better.

In practice, then, the easiest way to implement absorbing layers is to make an array that's the size of the model, build the various ramps in a few dozen cells towards each edges of the domain (taking the maximum value from the various ramps in each cell where the ramps overlap), and then multiply through each cell by its velocity model, and by $\delta t$. (We can build all the ramps to go from zero to one initially, and then scale everything appropriately afterwards.)

Also, it turns out we should divide by the grid-spacing, $\delta x$, to ensure the $e^{-\alpha ct}$decay has *dimensionless* exponential factor $\alpha ct$ – i.e. we end up building $\alpha$ instead as a dimensionless object, that does not depend upon our domain length-scales. So the decay at each time-step is based around factor of $e^{-\alpha(c\delta t/\delta x)}$ instead.

See code in notebook for comparison regarding above points.

### Predictive absorbing boundary...

This works by predicting what the value will be at the edge point(s) at the next step, because we know what the current value is at that point, and at the point just inside, and we know that the wave should be propagating out of the domain at some known speed (since we know the model properties).

By assuming a simple linear relation between the edge point and the point just inside, and 'shifting' that line across by the crossing factor (i.e. how far the wave should move during the step), we can 'predict' what the new value should be at the edge point.

$$\text{This looks like:}\quad
u(0,t+\delta t)\ \approx\ (1-C_0)\ u(0,t)\ +\ C_0\ u(\delta x,t)\quad
\text{where: }\ C_0 = \frac{c(0)\ \delta t}{\delta x}\ \text{ is 'crossing factor' near $x$=0}$$

$$\text{And for other edge:}\quad
u(L,t+\delta t)\ \approx\ (1-C_L)\ u(L,t)\ +\ C_L\ u(L-\delta x,t)\quad
\text{where: }\ C_L = \frac{c(L)\ \delta t}{\delta x} \text{ (i.e. near $x$=$L$)}$$

----
### Some potential propagation kernel optimisations...

Replacing loop over space with 'vectorised' versions. (This will be important for speed when it comes to 2d.)

So instead of using `for ix in range(1,nx-1)` with `u_nxt[ix] = ...` to loop over (almost) the whole array, we'll just use a single line that works on (almost) the whole array in one go: `u_nxt[1:-1] = ...`

(Will only do these next two if we're doing well for time...)

**>>> Optimisation: reduce to two wavefields instead of three**  
(i.e. avoid having both of `u_prv` and `u_nxt` by overwriting straight into `u_prv`)

**>>> More optimisation: avoid copying between wavefields by alternating**  
(Different ways... e.g. define propagation step as a function that takes two wavefields, and alternate order in call for each step)

----
### Fourth order stencil (in space)
(Probably won't get as far as this...)

$$f(x_0+\delta x) = f(x_0)+{\frac {f'(x_0)}{1!}}\delta x+{\frac {f''(x_0)}{2!}}\delta x^{2}+
{\frac {f'''(x_0)}{3!}}\delta x^{3}+{\frac {f''''(x_0)}{4!}}\delta x^{4}+
{\frac {f'''''(x_0)}{5!}}\delta x^{5}+O(\delta x^6)$$

$$f(x_0-\delta x) = f(x_0)-{\frac {f'(x_0)}{1!}}\delta x+{\frac {f''(x_0)}{2!}}\delta x^{2}-
{\frac {f'''(x_0)}{3!}}\delta x^{3}+{\frac {f''''(x_0)}{4!}}\delta x^{4}+
{\frac {f'''''(x_0)}{5!}}\delta x^{5}+O(\delta x^6)$$

$$f(x_0+2\delta x) = f(x_0)+{\frac {f'(x_0)}{1!}}(2\delta x)+{\frac {f''(x_0)}{2!}}(2\delta x)^{2}+
{\frac {f'''(x_0)}{3!}}(2\delta x)^{3}+{\frac {f''''(x_0)}{4!}}(2\delta x)^{4}+
{\frac {f'''''(x_0)}{5!}}(2\delta x)^{5}+O(\delta x^6)$$

$$f(x_0-2\delta x) = f(x_0)-{\frac {f'(x_0)}{1!}}(2\delta x)+{\frac {f''(x_0)}{2!}}(2\delta x)^{2}-
{\frac {f'''(x_0)}{3!}}(2\delta x)^{3}+{\frac {f''''(x_0)}{4!}}(2\delta x)^{4}-
{\frac {f'''''(x_0)}{5!}}(2\delta x)^{5}+O(\delta x^6)$$

$$\text{So:}\quad f(x_0+\delta x) + f(x_0-\delta x) = 2f(x_0) + f''(x_0)\delta x^{2} + 
\tfrac{1}{12}f''''(x_0)\delta x^{4} + O(\delta x^6)$$

$$\text{and:}\quad f(x_0+2\delta x) + f(x_0-2\delta x) = 2f(x_0) + 4f''(x_0)\delta x^{2} + 
\tfrac{4}{3}f''''(x_0)\delta x^{4} + O(\delta x^6)$$

$$\text{Eliminate $f''''(x_0)\delta x^4$:}\quad 16[f(x_0+\delta x) + f(x_0-\delta x)] - 
f(x_0+2\delta x) + f(x_0-2\delta x) = 
(32-2)f(x_0) + (16-4)f''(x_0)\delta x^{2} + O(\delta x^6)$$

$$\text{To give:}\quad f''(x_0) = \frac{-f(x_0-2\delta x)+16f(x_0-\delta x)-30f(x_0)+16f(x_0+\delta x)-f(x_0+2\delta x)}{12\delta x^2}\ +\ O(\delta x^4)$$

$$\text{We can now use that to replace the $\frac{\partial^2 u^\tau(x)}{\partial x^2}$ term in the time-stepping equation we found earlier:}$$

$$(1+q)\ u^{\tau+1}(x)\approx
\delta t^2c(x)^2\left(\frac{-u^\tau(x\text{–}2\delta x)+16u^\tau(x\text{–}\delta x)-30u^\tau(x)+16u^\tau(x\text{+}\delta x)-u^\tau(x\text{+}2\delta x)}{12\delta x^2}\right)$$

$$\quad\quad\quad\quad+\ (2+q^2)\ u^\tau(x)-(1-q)\ u^{\tau-1}(x)$$

$$\text{i.e. our fourth-order space-time discretisation finally becomes:}$$

$$\quad(1+q)\ u^{\tau+1}_\xi=
\frac{\delta t^2c_\xi^2}{\delta x^2}\left(\frac{-u^\tau_{\xi-2}+16u^\tau_{\xi-1}-30u^\tau_\xi+16u^\tau_{\xi+1}-u^\tau_{\xi+2}}{12}\right)+
(2+q^2)\ u^\tau_\xi-(1-q)\ u^{\tau-1}_\xi$$

**>>> What will be a new issue here for discretisation at edges of domain?**  
$\quad\quad\quad\quad\quad\quad\quad\quad\quad$**– for both reflective and 'predictive' boundary conditions?**  
Answer: We had the loop over x-gridpoints ($\xi$ above) range from 1 to nx-2 because the 2nd-order stencil looks at $\xi$-1 and $\xi$+1.  
But the 4th-order one also looks at $\xi$-2 and $\xi$+2, so we can now only loop from 2 to nx-3, which means there are *two* points on each side that don't get evaluated during a propagation step (not just one on each side).  
Concentrating on just one side for now (the $x$=0 side), this leads to questions about the values to set for one of these two points on this side, since we only have a specific value (zero) at one point, $x$=0. –Should we add an extra point at $x$=$-\delta x$ on this left side (and $x$=$L$+$\delta x$ on the other)? And then what do we set for that point's value at each step?  
It also means we need to do 'prediction' for two gridpoints (rather than just one) on each side that has a predictive boundary.